# Compute trajectories for Greenland

In [ ]:
import numpy as np
import pylab as plt
import xarray as xr
from joblib import Parallel, delayed
import geopandas as gp
from pathlib import Path
from pism_ragis.trajectories import compute_trajectory, compute_perturbation, get_perturbed_velocities
from pism_ragis.processing import tqdm_joblib
import pandas as pd
from pyDOE import lhs
from scipy.stats.distributions import uniform
from tqdm.auto import tqdm

In [ ]:
data_url = Path("~/Google Drive/My Drive/data/ITS_LIVE/GRE_G0240_0000.nc")
ogr_url = Path("/Users/andy/Google Drive/My Drive/data/GreenlandFluxGatesAschwanden/greenland-flux-gates-jibneighbors.shp")

In [ ]:
n_draw_samples = 10
unif_sample = lhs(2, n_draw_samples)

In [ ]:
with tqdm_joblib(tqdm(desc="Processing perturbations", total=n_draw_samples, position=0, leave=True)) as progress_bar:
    all_perturb_glaciers = Parallel(n_jobs=10)(
        delayed(compute_perturbation)(data_url, ogr_url,
                                       perturbation=perturb, sample=unif_sample[perturb, :], 
                                       total_time=1_000, dt=1, reverse=True)
                for perturb in range(n_draw_samples))
    del progress_bar

all_perturb_glaciers = pd.concat(all_perturb_glaciers).reset_index(drop=True)

In [ ]:
from tqdm.auto import trange